In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install smart_open

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
from gensim import models
import gensim.downloader as api

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import nltk
import string
import tqdm

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/aiyappa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aiyappa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
wv = api.load('word2vec-google-news-300')

In [9]:
reviews_raw = pd.read_csv("data/original_data.csv")
reviews_raw[['Title', 'Text']] = reviews_raw[['Title', 'Text']].fillna('')

In [10]:
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09


In [11]:
reviews_raw.shape

(12854, 9)

In [12]:
def tokenize_text(raw_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))

    tokenized_str = simple_preprocess(raw_text)
    tokens = [i.lower() for i in tokenized_str if (i not in string.punctuation) and (i not in stop_words)]
    return tokens

In [13]:
reviews_raw['tokenized_title'] = reviews_raw.Title.apply(tokenize_text)
reviews_raw['tokenized_text'] = reviews_raw.Text.apply(tokenize_text)
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title,tokenized_text
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,...",[]
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]",[]
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union]","[watched, state, union, last, night, opinion]"
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]",[]
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew],[]


In [14]:
reviews_raw = reviews_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title,tokenized_text
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,...",[]
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]",[]
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union]","[watched, state, union, last, night, opinion]"
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]",[]
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew],[]


In [15]:
dictionary = corpora.Dictionary(reviews_raw.tokenized_title)
inverse_dictionary = {dictionary.get(key): key for key in dictionary.keys()}

corpus = [dictionary.doc2bow(entry) for entry in reviews_raw.tokenized_title]
tfidf = models.TfidfModel(corpus, smartirs='ntc')
word_to_tfidf = {dictionary.get(id): value for doc in tfidf[corpus] for id, value in doc}

In [16]:
vector_length = 300
f_vec = []

for doc in tfidf[corpus]:
    words = [[dictionary[id], freq] for id, freq in doc]
    sentence_feature = np.zeros(vector_length)
    word_count = 0
    for word in words:
        if word[0] in wv:
            word_count += 1
            word_feature = wv[word[0]] * word[1]
            sentence_feature += word_feature

    sentence_feature = sentence_feature/word_count

    f_vec.append(sentence_feature)

reviews_raw['feature_vector_title'] = f_vec

/tmp/ipykernel_11382/2293496076.py:14: RuntimeWarning: invalid value encountered in true_divide
  sentence_feature = sentence_feature/word_count


In [17]:
dictionary = corpora.Dictionary(reviews_raw.tokenized_text)
inverse_dictionary = {dictionary.get(key): key for key in dictionary.keys()}

corpus = [dictionary.doc2bow(entry) for entry in reviews_raw.tokenized_text]
tfidf = models.TfidfModel(corpus, smartirs='ntc')
word_to_tfidf = {dictionary.get(id): value for doc in tfidf[corpus] for id, value in doc}

In [18]:
f_vec = []

for doc in tfidf[corpus]:
    words = [[dictionary[id], freq] for id, freq in doc]
    sentence_feature = np.zeros(vector_length)
    word_count = 0
    for word in words:
        if word[0] in wv:
            word_count += 1
            word_feature = wv[word[0]] * word[1]
            sentence_feature += word_feature

    sentence_feature = sentence_feature/word_count

    f_vec.append(sentence_feature)

reviews_raw['feature_vector_text'] = f_vec

/tmp/ipykernel_11382/1499145625.py:13: RuntimeWarning: invalid value encountered in true_divide
  sentence_feature = sentence_feature/word_count


In [19]:
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title,tokenized_text,feature_vector_title,feature_vector_text
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,...",[],"[0.002659287355426285, 0.006900876941573288, 0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]",[],"[0.036494743544608355, 0.03322484794383248, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union]","[watched, state, union, last, night, opinion]","[-0.09261803608387709, -0.0984919685870409, 0....","[0.0020916512391219535, 0.0009749974124133587,..."
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]",[],"[0.081322249956429, 0.02632659173104912, 0.006...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew],[],"[0.0191650390625, -0.034423828125, -0.07373046...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [20]:
columns_title = ['f_title'+str(i) for i in range(1,301)]
columns_text = ['f_text'+str(i) for i in range(1,301)]
columns = columns_title + columns_text
columns.append('lean')

data_df_title = reviews_raw.feature_vector_title.apply(pd.Series)
data_df_text = reviews_raw.feature_vector_text.apply(pd.Series)

data_df_title.columns = columns_title
data_df_text.columns = columns_text

In [21]:
data_df_title.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_title291,f_title292,f_title293,f_title294,f_title295,f_title296,f_title297,f_title298,f_title299,f_title300
0,0.002659,0.006901,0.025999,0.039320,-0.038575,0.011260,0.034370,-0.019639,0.009125,0.007236,...,-0.007184,0.015120,-0.017277,0.007305,-0.025121,-0.002852,0.013483,-0.007855,0.006062,0.005490
1,0.036495,0.033225,0.040508,0.045869,0.000460,0.021309,-0.001907,-0.022034,0.057259,0.007977,...,-0.008520,-0.007675,-0.063252,0.010747,0.029191,-0.079689,-0.004550,-0.018919,-0.007102,-0.034628
2,-0.092618,-0.098492,0.002448,-0.005652,0.024227,-0.123088,0.006178,-0.044742,0.059487,-0.026446,...,-0.042310,-0.031728,-0.088857,-0.070869,0.008909,0.034924,0.007204,-0.032756,-0.069787,-0.025426
3,0.081322,0.026327,0.006852,0.073578,0.014910,0.031147,-0.000066,-0.003802,0.025104,0.024137,...,-0.027032,0.062796,-0.062686,-0.038190,-0.013858,0.031323,0.003322,-0.028917,0.025368,-0.055304
4,0.019165,-0.034424,-0.073730,-0.237305,0.341797,0.059082,-0.006622,-0.031128,-0.373047,0.183594,...,0.109375,0.136719,0.421875,-0.404297,-0.023193,0.248047,0.003265,0.048340,-0.186523,0.192383


In [22]:
data_df_text.head()

,f_text1,f_text2,f_text3,f_text4,f_text5,f_text6,f_text7,f_text8,f_text9,f_text10,...,f_text291,f_text292,f_text293,f_text294,f_text295,f_text296,f_text297,f_text298,f_text299,f_text300
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.002092,0.000975,0.016885,0.028339,-0.022482,-0.034713,0.011809,-0.057471,0.059017,0.02712,...,-0.018685,-0.027967,-0.026938,0.014202,-0.005317,-0.036469,-0.021324,-0.050806,-0.000435,-0.022806
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
merged_df = pd.concat([data_df_title, data_df_text], axis=1)
merged_df = merged_df.fillna(0)
merged_df['upvotes'] = reviews_raw['Score']
merged_df['num_comments'] = reviews_raw['Num of Comments']
merged_df.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_text293,f_text294,f_text295,f_text296,f_text297,f_text298,f_text299,f_text300,upvotes,num_comments
0,0.002659,0.006901,0.025999,0.039320,-0.038575,0.011260,0.034370,-0.019639,0.009125,0.007236,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0
1,0.036495,0.033225,0.040508,0.045869,0.000460,0.021309,-0.001907,-0.022034,0.057259,0.007977,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6,1
2,-0.092618,-0.098492,0.002448,-0.005652,0.024227,-0.123088,0.006178,-0.044742,0.059487,-0.026446,...,-0.026938,0.014202,-0.005317,-0.036469,-0.021324,-0.050806,-0.000435,-0.022806,1,1
3,0.081322,0.026327,0.006852,0.073578,0.014910,0.031147,-0.000066,-0.003802,0.025104,0.024137,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7,3
4,0.019165,-0.034424,-0.073730,-0.237305,0.341797,0.059082,-0.006622,-0.031128,-0.373047,0.183594,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6,1


In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
merged_df_scaled = pd.DataFrame(scaler.fit_transform(merged_df), columns=merged_df.columns)
merged_df_scaled.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_text293,f_text294,f_text295,f_text296,f_text297,f_text298,f_text299,f_text300,upvotes,num_comments
0,0.486932,0.448939,0.517243,0.305784,0.509402,0.531498,0.491230,0.565285,0.398517,0.452034,...,0.688995,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000040,0.000000
1,0.521034,0.479571,0.531781,0.311606,0.553618,0.540621,0.449492,0.562698,0.448710,0.453057,...,0.688995,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000239,0.000465
2,0.390905,0.326300,0.493647,0.265809,0.580541,0.409537,0.458793,0.538169,0.451033,0.405551,...,0.645001,0.486827,0.697899,0.587096,0.451446,0.509188,0.362672,0.375478,0.000040,0.000465
3,0.566215,0.471544,0.498059,0.336236,0.569987,0.549552,0.451610,0.582391,0.415180,0.475359,...,0.688995,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000279,0.001395
4,0.503568,0.400852,0.417319,0.059896,0.940265,0.574911,0.444066,0.552874,0.000000,0.695418,...,0.688995,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000239,0.000465


In [27]:
merged_df_scaled['lean'] = reviews_raw['Political Lean']
merged_df_scaled.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_text294,f_text295,f_text296,f_text297,f_text298,f_text299,f_text300,upvotes,num_comments,lean
0,0.486932,0.448939,0.517243,0.305784,0.509402,0.531498,0.491230,0.565285,0.398517,0.452034,...,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000040,0.000000,Liberal
1,0.521034,0.479571,0.531781,0.311606,0.553618,0.540621,0.449492,0.562698,0.448710,0.453057,...,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000239,0.000465,Liberal
2,0.390905,0.326300,0.493647,0.265809,0.580541,0.409537,0.458793,0.538169,0.451033,0.405551,...,0.486827,0.697899,0.587096,0.451446,0.509188,0.362672,0.375478,0.000040,0.000465,Liberal
3,0.566215,0.471544,0.498059,0.336236,0.569987,0.549552,0.451610,0.582391,0.415180,0.475359,...,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000279,0.001395,Liberal
4,0.503568,0.400852,0.417319,0.059896,0.940265,0.574911,0.444066,0.552874,0.000000,0.695418,...,0.464174,0.709257,0.672498,0.474576,0.616016,0.363636,0.447562,0.000239,0.000465,Liberal


In [28]:
merged_df_scaled.shape

(12854, 603)

In [29]:
merged_df_scaled.to_csv('data/w2vecscaled.csv', index=False)